In [ ]:
# IUCN Data wrangling and visualization project
# Jupyter Notebook for data Wrangling and Visualization

# Step 1: Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt

# Define folder locations (makes it easy to reuse/change later)
DATA_DIR = "data/"
FIG_DIR = "figures/"

# Step 2: Read original CSV files into pandas DataFrames
# These files were exported from the Marine Mammals Global Species Database spreadsheet
hierarchy_df = pd.read_csv(DATA_DIR + 'IUCNHierarchyAndExtinction.csv')
country_occurrences_df = pd.read_csv(DATA_DIR + 'IUCNSpeciesCountryOccurrences.csv')

# Step 3: Create a new table for families
families_df = hierarchy_df[['family']].drop_duplicates().dropna()
families_df = families_df.rename(columns={'family': 'family_name'})

# Step 4: Create a table for genus information
genus_df = hierarchy_df[['genus', 'family']].drop_duplicates().dropna()
genus_df = genus_df.rename(columns={'genus': 'genus_name', 'family': 'family_name'})

# Step 5: Create a table for species
species_df = hierarchy_df[['taxonid', 'genus', 'species', 'friendly_name', 'category']].drop_duplicates().dropna()
species_df = species_df.rename(columns={
    'genus': 'genus_name',
    'species': 'scientific_name'
})

# Step 6: Create a table for country occurrences
country_occurrences_clean_df = country_occurrences_df[['taxonid', 'countryoccurrencelookup_value']].drop_duplicates().dropna()
country_occurrences_clean_df = country_occurrences_clean_df.rename(columns={'countryoccurrencelookup_value': 'country'})

# Step 7: Save the cleaned tables as new CSV files (into data/ folder)
families_df.to_csv(DATA_DIR + 'families.csv', index=False)
genus_df.to_csv(DATA_DIR + 'genus.csv', index=False)
species_df.to_csv(DATA_DIR + 'species.csv', index=False)
country_occurrences_clean_df.to_csv(DATA_DIR + 'country_occurrences.csv', index=False)

# Step 8: Load the SQL query result for threatened species per family
# This was generated using UPPER(TRIM(REPLACE(REPLACE(s.category, '\r', ''), '\n', ''))) IN ('CR', 'EN', 'VU')
threatened_df = pd.read_csv(DATA_DIR + 'threatened_species_by_family.csv')

# Step 9: Clean family names of hidden characters (like \r, \n)
threatened_df['family_name'] = threatened_df['family_name'].str.replace(r'[\r\n]+', '', regex=True).str.strip()

# Step 10: Sort the result for better readability
threatened_df = threatened_df.sort_values('threatened_species_count', ascending=False)

# Step 11: Plot the result as a Pie Chart with raw counts and save to file
plt.figure(figsize=(10, 8))
labels_with_counts = [f"{name} ({count})" for name, count in zip(threatened_df['family_name'], threatened_df['threatened_species_count'])]
plt.pie(threatened_df['threatened_species_count'], labels=labels_with_counts, autopct='%1.1f%%')
plt.title('Distribution of Threatened Species per Family (CR, EN, VU)')
plt.axis('equal')
plt.tight_layout()
plt.savefig(FIG_DIR + 'threatened_species_pie_chart.png')  # Save the chart as a PNG file
plt.show()



: 